In [1]:
import json
import requests
import sys
from datetime import datetime, time, timedelta
sys.path.insert(0, '../../../commons/')
sys.path.insert(0, '../../../virasana/')

VIRASANA_URL = "https://ajna.labin.rf08.srf/virasana/"

In [2]:
diaapesquisar = datetime.today() - timedelta(days=2)
datainicial = datetime.strftime(datetime.combine(diaapesquisar, time.min), '%Y-%m-%d  %H:%M:%S')
datafinal = datetime.strftime(datetime.combine(diaapesquisar, time.max), '%Y-%m-%d %H:%M:%S')
print(datainicial, datafinal)

2019-11-10  00:00:00 2019-11-10 23:59:59


In [36]:
params = {'query': 
          { 'metadata.dataescaneamento': {'$gte': datainicial, '$lte': datafinal},
            'metadata.contentType': 'image/jpeg',
            'metadata.carga': {'$exists': False}
          },
          'projection': 
          {'metadata.numeroinformado': 1,
           'metadata.dataescaneamento': 1}
         }

r = requests.post(VIRASANA_URL + "grid_data", json=params, verify=False)
listacc = list(r.json())
print(len(listacc))


864


c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [37]:
listanumerocc = set([item['metadata']['numeroinformado'] for item in listacc])
print(len(listanumerocc))

848


In [38]:
import pymysql

# db = pymysql.connect(SQL_URI)




In [39]:
SQL = 'SELECT * FROM conteinervazioresumo c ' + \
      'inner join manifestosresumo m on c.manifesto = m.numero ' + \
      'where tipoTrafego = "5" AND ' + \
      'c.idConteinerVazio IN ("' + '", "'. join(listanumerocc) + '")'


In [40]:
connection = pymysql.connect(host='mysql.labin.rf08.srf',
                             user='admmerc',
                             password='XXX',
                             db='dbmercante',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        cursor.execute(SQL)
        result = cursor.fetchall()
finally:
    connection.close()

In [41]:
print(len(result))

35
